# Pytorch VGG-16 Fine tuning
Using PyTorch, I fine-tuned the learned weights for the VGG16 network architecture.
I'm sure there are a lot of things that could be improved, but I hope this will be helpful for everyone implementing this in PyTorch.
Please let me know if there's anything I should fix!

In [ ]:
import glob
import os.path as osp

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tqdm.notebook import tqdm
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

## Import Data

In [ ]:
df_train = pd.read_csv("/kaggle/input/plant-pathology-2021-fgvc8/train.csv")
df_sub = pd.read_csv("/kaggle/input/plant-pathology-2021-fgvc8/sample_submission.csv")

## Check train

In [ ]:
display(df_train)
display(df_train.labels.value_counts())

In [ ]:
d_set = set()
for k in df_train.labels.unique():
    d_set = d_set | set(k.split(" "))
print(f"num of labels: {len(d_set)}  {d_set}")

## Check submission

In [ ]:
display(df_sub)

## Label encoding

In [ ]:
def to_label(df):
    """
    Function for Label encoding.
    """
    le = LabelEncoder()
    df["labels_n"] = le.fit_transform(df.labels.values)
    return df

df_train = to_label(df_train)
df_labels_idx = df_train.loc[df_train.duplicated(["labels", "labels_n"])==False]\
                [["labels_n", "labels"]].set_index("labels_n").sort_index()
display(df_labels_idx)

## Image path

In [ ]:
def make_datapath_list(phase="train", val_size=0.25):
    """
    Function to create a PATH to the data.
    
    Parameters
    ----------
    phase : 'train' or 'val' or 'test'
        Specify whether to use Train data or test data.
    val_size : float
        Ratio of validation data to train data
        
    Returns
    -------
    path_lsit : list
        A list containing the PATH to the data.
    """
    
    if phase in ["train", "val"]:
        phase_path = "train_images"
    elif phase in ["test"]:
        phase_path = "test_images"
    else:
        print(f"{phase} not in path")
    rootpath = "/kaggle/input/plant-pathology-2021-fgvc8/"
    target_path = osp.join(rootpath+phase_path+"/*.jpg")
    path_list = []
    
    for path in glob.glob(target_path):
        path_list.append(path)
        
    if phase in ["train", "val"]:
        train, val = train_test_split(path_list, test_size=val_size, random_state=0, shuffle=True)
        if phase == "train":
            path_list = train
        else:
            path_list = val
    
    return path_list

In [ ]:
train_list = make_datapath_list(phase="train")
print(f"train data length : {len(train_list)}")
val_list = make_datapath_list(phase="val")
print(f"validation data length : {len(val_list)}")
test_list = make_datapath_list(phase="test")
print(f"test data length : {len(test_list)}")

## Dataset

In [ ]:
class ImageTransform():
    """
    Class for image preprocessing.
    
    Attributes
    ----------
    resize : int
        224
    mean : (R, G, B)
        Average value for each color channel
    std : (R, G, B)
        Standard deviation for each color channel
    """
    
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'test': transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
    
    def __call__(self, img, phase="train"):
        """
        Parameters
        ----------
        phase: 'train' or 'val' or 'test'
            Specify the mode of preprocessing
        """
        return self.data_transform[phase](img)

In [ ]:
# test for ImageTransform Class 
image_file_path = '/kaggle/input/plant-pathology-2021-fgvc8/train_images/800113bb65efe69e.jpg'
img = Image.open(image_file_path)

plt.imshow(img)
plt.show()

size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

transform = ImageTransform(size, mean, std)
img_transformed = transform(img, phase='train')
print(img_transformed.shape)

img_transformed = img_transformed.numpy().transpose([1, 2, 0])
img_transformed = np.clip(img_transformed, 0, 1)
plt.imshow(img_transformed)
plt.show()

In [ ]:
class PlantDataset(data.Dataset):
    """
    Class to create a Dataset
    
    Attributes
    ----------
    df_train : DataFrame
        DataFrame containing the image labels.
    file_list : list
        A list containing the paths to the images
    transform : object
        Instance of the preprocessing class (ImageTransform)
    phase : 'train' or 'val' or 'test'
        Specify whether to use train, validation, or test
    """
    def __init__(self, df_train, file_list, transform=None, phase='train'):
        self.df_train = df_train
        self.df_labels_idx = df_labels_idx
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        """
        Returns the number of images.
        """
        return len(self.file_list)
    
    def __getitem__(self, index):
        """
        Get data in Tensor format and labels of preprocessed images.
        """
        #print(index)
        
        # Load the index number image.
        img_path = self.file_list[index]
        img = Image.open(img_path)
        
        # Preprocessing images
        img_transformed = self.transform(img, self.phase)
        
        # image name
        image_name = img_path[-20:]
        
        # Extract the labels
        if self.phase in ["train", "val"]:
            label = df_train.loc[df_train["image"]==image_name]["labels_n"].values[0]
        elif self.phase in ["test"]:
            label = -1
        
        return img_transformed, label, image_name

In [ ]:
train_dataset = PlantDataset(df_train, train_list, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = PlantDataset(df_train, val_list, transform=ImageTransform(size, mean, std), phase='val')
test_dataset = PlantDataset(df_train, test_list, transform=ImageTransform(size, mean, std), phase='test')

index = 0

print("【train dataset】")
print(f"img num : {train_dataset.__len__()}")
print(f"img : {train_dataset.__getitem__(index)[0].size()}")
print(f"label : {train_dataset.__getitem__(index)[1]}")
print(f"image name : {train_dataset.__getitem__(index)[2]}")

print("\n【validation dataset】")
print(f"img num : {val_dataset.__len__()}")
print(f"img : {val_dataset.__getitem__(index)[0].size()}")
print(f"label : {val_dataset.__getitem__(index)[1]}")
print(f"image name : {val_dataset.__getitem__(index)[2]}")

print("\n【test dataset】")
print(f"img num : {test_dataset.__len__()}")
print(f"img : {test_dataset.__getitem__(index)[0].size()}")
print(f"label : {test_dataset.__getitem__(index)[1]}")
print(f"image name : {test_dataset.__getitem__(index)[2]}")

## Dataloader

In [ ]:
batch_size = 128

# Create DataLoader
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# to Dictionary
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader, "test": test_dataloader}

# Operation check
#batch_iterator = iter(dataloaders_dict["train"])
#inputs, labels = next(batch_iterator)
#print(inputs.size())  # torch.Size([3, 3, 224, 224]) : [batch_size, Channel, H, W]
#print(labels)

## Network model

In [ ]:
# Load the learned VGG-16 model.

# Create an instance of the VGG-16 model
use_pretrained = False
net = models.vgg16(pretrained=use_pretrained)

#save_path = "/kaggle/working/vgg16_pretrained.h"
#torch.save(net.state_dict(), save_path)

load_path = "/kaggle/input/d/kuboko/plantpathology2021/vgg16_pretrained.h"
if torch.cuda.is_available():
    load_weights = torch.load(load_path)
    net.load_state_dict(load_weights)
else:
    load_weights = torch.load(load_path, map_location={"cuda:0": "cpu"})
    net.load_state_dict(load_weights)

# Replace the output unit of the last output layer of the VGG-16 model.
# out_features 1000 to 12
net.classifier[6] = nn.Linear(in_features=4096, out_features=12)

# Set to training mode.
net.train()

## Loss function

In [ ]:
criterion = nn.CrossEntropyLoss()

## Optimizer

In [ ]:
# Store the parameters to be learned by finetuning in the variable params_to_update.
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# Specify the parameter name of the layer to be trained.
update_param_names_1 = ["features.24.weight", "features.24.bias", "features.26.weight", "features.26.bias", "features.28.weight", "features.28.bias"]
update_param_names_2 = ["classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

for name, param in net.named_parameters():
    if name in update_param_names_1:
        param.requires_grad = True
        params_to_update_1.append(param)
        print(f"Store in params_to_update_1 : {name}")
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print(f"Store in params_to_update_2 : {name}")
    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print(f"Store in params_to_update_3 : {name}")
    else:
        param.requires_grad = False
        print(f"Parameters not to be learned :  {name}")

In [ ]:
# Set Optimizer
optimizer = optim.SGD([
    {"params": params_to_update_1, "lr": 1e-4},
    {"params": params_to_update_2, "lr": 5e-4},
    {"params": params_to_update_3, "lr": 1e-3}
], momentum=0.9)

## Function for model training

In [ ]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    """
    Function for training the model.
    
    Parameters
    ----------
    net: object
    dataloaders_dict: dictionary
    criterion: object
    optimizer: object
    num_epochs: int
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Devices to be used : {device}")
    net.to(device)
    torch.backends.cudnn.benchmark = True
    # loop for epoch
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1} / {num_epochs}")
        print("-------------------------------")
        for phase in ["train", "val"]:
            if phase == "train":
                net.train()
            else:
                net.eval()
            epoch_loss = 0.0
            epoch_corrects = 0
            #if (epoch == 0) and (phase == "train"):
                #continue
            for inputs, labels, _ in tqdm(dataloaders_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == "train"):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

# Start training 

In [ ]:
num_epochs = 4
# train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

## save weights

In [ ]:
#save_path = "./vgg16_fine_tuning_v1.h"
#torch.save(net.state_dict(), save_path)

## load weights

In [ ]:
load_path = "/kaggle/input/d/kuboko/plantpathology2021/vgg16_fine_tuning_v1.h"
if torch.cuda.is_available():
    load_weights = torch.load(load_path)
    net.load_state_dict(load_weights)
else:
    load_weights = torch.load(load_path, map_location={"cuda:0": "cpu"})
    net.load_state_dict(load_weights)

In [ ]:
net

## inference

In [ ]:
#batch_iterator = iter(dataloaders_dict["val"])
#inputs, labels, image_name = next(batch_iterator)
#print(inputs.size())  # torch.Size([3, 3, 224, 224]) : [batch_size, Channel, H, W]
#print(labels)

In [ ]:
class PlantPredictor():
    """
    Class for predicting labels from output results
    
    Attributes
    ----------
    df_labels_idx: DataFrame
        DataFrame that associates INDEX with a label name
    """
    
    def __init__(self, net, df_labels_idx, dataloaders_dict):
        self.net = net
        self.df_labels_idx = df_labels_idx
        self.dataloaders_dict = dataloaders_dict
        self.df_submit = pd.DataFrame()
        
    
    def __predict_max(self, out):
        """
        Get the label name with the highest probability.
        
        Parameters
        ----------
        predicted_label_name: str
            Name of the label with the highest prediction probability
        """
        maxid = np.argmax(out.detach().numpy(), axis=1)
        df_predicted_label_name = self.df_labels_idx.iloc[maxid]
        
        return df_predicted_label_name
    
    def inference(self):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(f"Devices to be used : {device}")
        df_pred_list = []
        for inputs, _, image_name in tqdm(self.dataloaders_dict['test']):
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            self.net.to(device)
            inputs = inputs.to(device)
            out = self.net(inputs)
            device = torch.device("cpu")
            out = out.to(device)
            df_pred = self.__predict_max(out).reset_index(drop=True)
            df_pred["image"] = image_name
            df_pred_list.append(df_pred)
            
        self.df_submit = pd.concat(df_pred_list, axis=0)
        self.df_submit = self.df_submit[["image", "labels"]].reset_index(drop=True)
            
        

In [ ]:
predictor = PlantPredictor(net, df_labels_idx, dataloaders_dict)
predictor.inference()
#df_pred = predictor.predict_max(out)

#df_sub.labels = df_pred.labels.reset_index(drop=True)
#display(df_pred)
#display(df_sub)

In [ ]:
df_submit = predictor.df_submit.copy()

## Submit

In [ ]:
df_submit.to_csv("/kaggle/working/submission.csv", index=False)